In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import geopandas as gpd
from shapely.geometry import box
import os

c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def def_dims(nc_file):
    lons = np.array(nc_file.variables[[name for name in nc_file.indexes if ('lon' in name.lower()) | ('eas' in name.lower()) | ('wes' in name.lower()) | ('x' in name.lower())][0]][:])
    lats = np.array(nc_file.variables[[name for name in nc_file.indexes if ('lat' in name.lower()) | ('nor' in name.lower()) | ('sou' in name.lower()) | ('y' in name.lower())][0]][:])
    dif_lon = np.abs(lons[0] - lons[1], dtype= np.float64)
    dif_lat = np.abs(lats[0] - lats[1], dtype= np.float64)
    return dif_lon, dif_lat, lons, lats

In [3]:
aod = xr.open_dataset(r"C:\Users\user\OneDrive\Desktop\gdal trials\AOD nc\2022.nc").drop('spatial_ref')
folder = './Era5 Land/'
files = os.listdir(folder)
era5 = [xr.open_dataset(folder + file).coarsen(valid_time=24).mean() for file in files]
era5 = xr.merge(era5)

In [4]:
lon_dim = [name for name in aod.indexes if ('lon' in name.lower()) | ('eas' in name.lower()) | ('wes' in name.lower()) | ('x' in name.lower())][0]
lat_dim = [name for name in aod.indexes if ('lat' in name.lower()) | ('nor' in name.lower()) | ('sou' in name.lower()) | ('y' in name.lower())][0]
aod = aod.rename({lat_dim: 'latitude'})
aod = aod.rename({lon_dim: 'longitude'})

In [5]:
aod

<xarray.Dataset>
Dimensions:    (time: 365, latitude: 121, longitude: 191)
Coordinates:
  * longitude  (longitude) float64 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
  * latitude   (latitude) float64 32.2 32.19 32.18 32.17 ... 31.02 31.01 31.0
  * time       (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2022-12-31
Data variables:
    AOD055     (time, latitude, longitude) float32 ...

In [6]:
era5

<xarray.Dataset>
Dimensions:     (valid_time: 1523, latitude: 15, longitude: 22)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 2020-12-01T11:30:00 ... 2025-01-3...
  * latitude    (latitude) float64 32.3 32.2 32.1 32.0 ... 31.2 31.1 31.0 30.9
  * longitude   (longitude) float64 35.4 35.5 35.6 35.7 ... 37.2 37.3 37.4 37.5
    number      int64 0
    expver      (valid_time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
Data variables:
    d2m         (valid_time, latitude, longitude) float32 282.7 283.3 ... 272.4
    t2m         (valid_time, latitude, longitude) float32 287.9 289.8 ... 285.1
    u10         (valid_time, latitude, longitude) float32 -0.01531 ... 2.127
    v10         (valid_time, latitude, longitude) float32 0.1992 ... -0.7484
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-24T23:22 GRIB to CDM+CF via cfgrib-0.9.1...

In [38]:
'''dif_lon, dif_lat, lons, lats = def_dims(aod)
dss = []
crs = 'epsg:4326'
regridding_factor = 20
for i in range(len(lons)):
    for j in range(len(lats)):
        boxx = create_box(lons[i], lats[j], dif_lon, dif_lat, crs = crs)
        df = extracting_and_averaging_polygon(boxx, era5, regridding_factor, crs = crs)
        df['longitude'] = lons[i]
        df['latitude'] = lats[j]
        df.set_index(['time', 'longitude', 'latitude'], inplace=True)
        
        ds = df.to_xarray()
        del df, boxx
        if len(dss) == 0:
            dss = ds
        else:
            dss = xr.merge([dss, ds])
            del ds'''

"dif_lon, dif_lat, lons, lats = def_dims(aod)\ndss = []\ncrs = 'epsg:4326'\nregridding_factor = 20\nfor i in range(len(lons)):\n    for j in range(len(lats)):\n        boxx = create_box(lons[i], lats[j], dif_lon, dif_lat, crs = crs)\n        df = extracting_and_averaging_polygon(boxx, era5, regridding_factor, crs = crs)\n        df['longitude'] = lons[i]\n        df['latitude'] = lats[j]\n        df.set_index(['time', 'longitude', 'latitude'], inplace=True)\n        \n        ds = df.to_xarray()\n        del df, boxx\n        if len(dss) == 0:\n            dss = ds\n        else:\n            dss = xr.merge([dss, ds])\n            del ds"

In [7]:
dif_lon, dif_lat, lons, lats = def_dims(aod)
del dif_lat, dif_lon

df = []
for i in range(len(lons)):
    for j in range(len(lats)):
        dat = era5.sel(latitude=lats[j], longitude=lons[i], method='nearest').to_pandas().drop(['number', 'expver'], axis=1).reset_index()
        dat['time'] = pd.DatetimeIndex(dat['valid_time'].dt.date)
        dat['latitude'] = lats[j]
        dat['longitude'] = lons[i]
        dat.drop('valid_time', axis=1, inplace=True)
        df.append(dat)

In [8]:
era5_as = pd.concat(df).set_index(['time', 'longitude', 'latitude']).to_xarray()

In [18]:
era5_as

<xarray.Dataset>
Dimensions:    (time: 1523, longitude: 191, latitude: 121)
Coordinates:
  * time       (time) datetime64[ns] 2020-12-01 2020-12-02 ... 2025-01-31
  * longitude  (longitude) float64 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
  * latitude   (latitude) float64 31.0 31.01 31.02 31.03 ... 32.18 32.19 32.2
Data variables:
    d2m        (time, longitude, latitude) float32 283.2 283.2 ... 272.4 272.4
    t2m        (time, longitude, latitude) float32 291.3 291.3 ... 283.3 283.3
    u10        (time, longitude, latitude) float32 0.06851 0.06851 ... 3.489
    v10        (time, longitude, latitude) float32 -0.163 -0.163 ... 0.8076

In [19]:
# Number of chunks
n = 16

# Get indices to split along the time dimension
time_len = era5_as.dims["time"]
splits = np.array_split(np.arange(time_len), n)

# Save each chunk to a separate file
for i, idx in enumerate(splits):
    ds_subset = era5_as.isel(time=idx)
    ds_subset.to_netcdf(f"./Era5 Land Assimilated/Era5_land_{i+1}.nc")

In [23]:
folder = './CAMS Reanalysis AOD/'
files = os.listdir(folder)
cams_aod = [xr.open_dataset(folder + file).coarsen(forecast_reference_time=2).mean() for file in files]
cams_aod = xr.merge(cams_aod)

In [29]:
cams_aod

<xarray.Dataset>
Dimensions:                  (forecast_reference_time: 1503,
                              forecast_period: 1, latitude: 4, longitude: 6)
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 00:00:00
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2020-12...
  * latitude                 (latitude) float64 32.1 31.7 31.3 30.9
  * longitude                (longitude) float64 35.4 35.8 36.2 36.6 37.0 37.4
Data variables:
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] ...
    aod550                   (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    duaod550                 (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-28T15:35 GRIB to CDM+CF via cfgrib-0.9.1...

In [30]:
time_dim = [name for name in cams_aod.indexes if ('time' in name.lower()) | ('date' in name.lower()) | ('year' in name.lower()) | ('month' in name.lower())][0]
time_dim

'forecast_reference_time'

In [33]:
cams_aod[time_dim] = pd.DatetimeIndex(cams_aod[time_dim].dt.date)

In [34]:
cams_aod = cams_aod.rename({time_dim: 'time'})

In [40]:
cams_aod = cams_aod.drop('valid_time')

In [49]:
cams_aod = cams_aod.to_dataframe().reset_index().drop('forecast_period', axis=1).set_index(['time','longitude','latitude']).to_xarray()

In [50]:
cams_aod

<xarray.Dataset>
Dimensions:    (time: 1503, longitude: 6, latitude: 4)
Coordinates:
  * time       (time) datetime64[ns] 2020-12-10 2020-12-11 ... 2025-01-20
  * longitude  (longitude) float64 35.4 35.8 36.2 36.6 37.0 37.4
  * latitude   (latitude) float64 30.9 31.3 31.7 32.1
Data variables:
    aod550     (time, longitude, latitude) float32 0.1457 0.1239 ... 0.0483
    duaod550   (time, longitude, latitude) float32 0.02347 0.01908 ... 0.007282

In [51]:
dif_lon, dif_lat, lons, lats = def_dims(aod)
del dif_lat, dif_lon

df = []
for i in range(len(lons)):
    for j in range(len(lats)):
        dat = cams_aod.sel(latitude=lats[j], longitude=lons[i], method='nearest').to_pandas().reset_index()
        #dat['time'] = pd.DatetimeIndex(dat['valid_time'].dt.date)
        dat['latitude'] = lats[j]
        dat['longitude'] = lons[i]
        #dat.drop('valid_time', axis=1, inplace=True)
        df.append(dat)

In [57]:
cams_aod_as = pd.concat(df).set_index(['time', 'longitude', 'latitude']).to_xarray()

In [60]:
xr.merge([cams_aod_as, aod])

<xarray.Dataset>
Dimensions:    (time: 1503, longitude: 191, latitude: 121)
Coordinates:
  * time       (time) datetime64[ns] 2020-12-10 2020-12-11 ... 2025-01-20
  * longitude  (longitude) float64 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
  * latitude   (latitude) float64 31.0 31.01 31.02 31.03 ... 32.18 32.19 32.2
Data variables:
    aod550     (time, longitude, latitude) float32 0.1457 0.1457 ... 0.0483
    duaod550   (time, longitude, latitude) float32 0.02347 0.02347 ... 0.007282
    AOD055     (time, latitude, longitude) float32 nan nan nan ... nan nan nan

In [58]:
# Number of chunks
n = 8

# Get indices to split along the time dimension
time_len = cams_aod_as.dims["time"]
splits = np.array_split(np.arange(time_len), n)

# Save each chunk to a separate file
for i, idx in enumerate(splits):
    ds_subset = cams_aod_as.isel(time=idx)
    ds_subset.to_netcdf(f"./CAMS Reanalysis AOD Assimilated/cams_aod_{i+1}.nc")